In [11]:
import json
import boto3
import pickle
import pyarrow

import numpy as np
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta

from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction import DictVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

# import xgboost as xgb
from xgboost import XGBRegressor

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval
from hyperopt.pyll import scope

from prefect import task, flow
from prefect.task_runners import SequentialTaskRunner

from prefect.deployments import Deployment   
from prefect.orion.schemas.schedules import IntervalSchedule, CronSchedule
# from prefect.flow_runners import SubprocessFlowRunner

# mlflow.set_tracking_uri("http://127.0.0.1:5000/")


def read_file(key, bucket='kkr-mlops-zoomcamp'):

    session = boto3.session.Session()
    s3 = session.client(
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net',
        region_name='ru-central1',
        # aws_access_key_id = "id",
        # aws_secret_access_key = "key")
    )
    obj = s3.get_object(Bucket=bucket, Key=key)

    data = pd.read_csv(obj['Body'], sep=",")

    return data


@task
def load_data(current_date = "2015-5-17", periods = 1):
    
    dt_current = datetime.strptime(current_date, "%Y-%m-%d")
    
    if periods == 1:
        date_file = dt_current + relativedelta(months = - 1)
        print(f"Getting TEST data for {date_file.year}-{date_file.month} period")
        test_data = read_file(key = f"datasets/car-prices-{date_file.year}-{date_file.month}.csv")

        return test_data

    else:
        train_data = pd.DataFrame()
        for i in range(periods+1, 1, -1):
            date_file = dt_current + relativedelta(months = - i)
            try:
                data = read_file(key = f"datasets/car-prices-{date_file.year}-{date_file.month}.csv")
                print(f"Getting TRAIN data for {date_file.year}-{date_file.month} period")
            except:
                print(f"Cannot find file car-prices-{date_file.year}-{date_file.month}.csv",
                    "using blank")
                data = None
                
            train_data = pd.concat([train_data, data])
        
        return train_data


@task
def na_filter(data):
    work_data = data.copy()
    non_type = work_data[data['make'].isna() | data['model'].isna() | data['trim'].isna()].index
    work_data.drop(non_type, axis=0, inplace=True)

    y = work_data.pop('sellingprice')

    return work_data, y


class FeaturesModifier:
    def __init__(self, columns):
        self.columns = columns

    def fit(self, work_data, _ = None):
        return self

    def transform(self, work_data, _ = None):

        work_data = pd.DataFrame(work_data, columns = self.columns)
        work_data['make_model_trim'] = work_data['make'] + '_'  + work_data['model'] + '_' + work_data['trim']
        work_data['year'] = work_data['year'].astype('str')
        
        cat_cols = ['year', 'make_model_trim', 'body', 'transmission', 'color', 'interior']
        num_cols = ['condition', 'odometer', 'mmr']

        X = work_data[cat_cols + num_cols].copy()
        X_dict = X.to_dict(orient = 'records')

        return X_dict

    def fit_transform(self, work_data, _ = None):
        return self.transform(work_data)


@task
def prepare_features(work_data, preprocessor = None):

    num_2_impute = ['condition', 'odometer', 'mmr']
    cat_2_impute = ['body', 'transmission']
    constant_2_impute = ['color', 'interior']
    others = ['year', 'make', 'model', 'trim']
    
    if not preprocessor:
        features_filler = ColumnTransformer([
            ('num_imputer', SimpleImputer(missing_values=np.nan, strategy='mean'), num_2_impute),
            ('cat_imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent'), cat_2_impute),
            ('cat_constant', SimpleImputer(missing_values=np.nan, strategy='most_frequent'), constant_2_impute),
            ('others', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='-1'), others )
            ]
        )

        fm = FeaturesModifier(columns = num_2_impute + cat_2_impute + constant_2_impute + others)

        dv = DictVectorizer() 

        preprocessor = Pipeline(steps = [
            ('filler', features_filler),
            ('modifier', fm),
            ('dict_vectorizer', dv)
        ])
        
        X = preprocessor.fit_transform(work_data)

    else:
        X = preprocessor.transform(work_data)

    return X, preprocessor


@task
def params_search(MLFLOW_TRACKING_URI, train, valid, y_train, y_valid, current_date, models):
    
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

    best_models = []

    for baseline in models:
        
        mlflow.set_experiment(f"{baseline.__name__}-models")
        search_space = models[baseline]

        def objective(params):

            with mlflow.start_run():
                mlflow.set_tag("baseline", f"{baseline.__name__}")
                mlflow.log_param("training-data", current_date[:7])
                mlflow.log_param("parameters", params)
                
                print('$$$ Serching for the best parameters... $$$')

                training_model = baseline(**params)
                training_model.fit(train, y_train)

                print('$$$ Predicting on the valid dataset... $$$')
                prediction_valid = training_model.predict(valid)
                rmse_valid = mean_squared_error(y_valid, prediction_valid, squared = False)

                print('$$$ RMSE on valid $$$', 
                    rmse_valid
                    )
                mlflow.log_metric('rmse', rmse_valid)
            

            return {'loss': rmse_valid, 'status': STATUS_OK}
        
        best_result = fmin(fn = objective,
                    space = search_space,
                    algo = tpe.suggest,
                    max_evals = 2, # int(2**(len(models[baseline].items())-2)), #3,
                    trials = Trials(),
                    ) 
        
        print("$$$ Best model $$$\n", baseline(**space_eval(search_space, best_result)))
        best_models.append(baseline(**space_eval(search_space, best_result)))

        mlflow.end_run()
    
    return best_models #ML_model(**space_eval(search_space, best_result))

@task
def train_best_models(MLFLOW_TRACKING_URI, best_models_experiment, train, y_train, X_valid, y_valid, X_test, y_test, preprocessor, models):

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow_client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

    best_pipelines = []

    mlflow.autolog()
    for model in models:
    
        experiment = mlflow.set_experiment(f"{model.__name__}-models")

        best_run = mlflow_client.search_runs(
                experiment_ids = experiment.experiment_id,
                run_view_type=ViewType.ACTIVE_ONLY,
                max_results = 2,
                order_by = ['metrics.rmse ASC']
            )
        
        print(f"$$$ Training {model.__name__} with best params $$$")

        mlflow.set_experiment(best_models_experiment) #"Auction-car-prices-best-models")
    
        with mlflow.start_run():
            
            mlflow.log_param("test_dataset", X_test["saledate"].max()[:7])

            best_params = json.loads(best_run[0].data.params['parameters'].replace("'", "\""))
            staged_model = model(**best_params).fit(train, y_train)
            
            pipeline = Pipeline(
                steps = [
                    ('preprocessor', preprocessor),
                    ('model', staged_model)
                ]
            )
            predict_valid = pipeline.predict(X_valid)
            rmse_valid = mean_squared_error(y_valid, predict_valid, squared = False)

            predict_test = pipeline.predict(X_test)
            rmse_test = mean_squared_error(y_test, predict_test, squared = False)

            mlflow.log_metric("rmse_valid", rmse_valid)
            mlflow.log_metric("rmse_test", rmse_test)
            mlflow.sklearn.log_model(pipeline, artifact_path='full-pipeline')
            
            best_pipelines.append((model.__name__, pipeline))

            print("$$$ {:} MODEL was saved as a RUN of {:} $$$".format(model.__name__, best_models_experiment))

            mlflow.end_run()

    return best_pipelines


@task
def model_to_registry(MLFLOW_TRACKING_URI, best_models_experiment, model_name):

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow_client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)
    
    experiment = mlflow.set_experiment(best_models_experiment) #'Auction-car-prices-best-models')

    best_model_run = mlflow_client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1,
        order_by=["metrics.rmse_test ASC"]
            
        )
    RUN_ID = best_model_run[0].info.run_id
    model_uri = "runs:/{:}/full-pipeline".format(RUN_ID)
   
    print(f"$$$ Registering model {model_name} $$$")
    registered_model = mlflow.register_model(
            model_uri=model_uri,
            name = model_name
        )
    print(f"$$$ Model RUN_ID {registered_model.run_id} was registered as version {registered_model.version} at {registered_model.current_stage} stage $$$")
    
    return registered_model


@task
def model_promotion(MLFLOW_TRACKING_URI, current_date, model_name, registered_model_version, to_stage):

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow_client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

    promoted_model = mlflow_client.transition_model_version_stage(
                                name = model_name,
                                version = registered_model_version,
                                stage = to_stage,
                                archive_existing_versions=True
                                )

    mlflow_client.update_model_version(
        name = model_name,
        version = registered_model_version,
        description=f'The model was promoted to {to_stage} {current_date}'
        )
    print(f"$$$ Model {model_name} version {registered_model_version} was promoted to {to_stage} {current_date} $$$")

    return promoted_model


def load_model(MLFLOW_TRACKING_URI, model_name, stage=None, version=None, run_id=None):
    
    mlflow_client = MlflowClient(MLFLOW_TRACKING_URI)
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

    versions = mlflow_client.get_latest_versions(
                name=model_name,
                stages=[stage]
                )

    try:
        model_uri = f"models:/{model_name}/{versions[0].current_stage}"
        model = mlflow.pyfunc.load_model(model_uri = model_uri)

        return model, versions[0].version
    except:
        print(f"$$$ There are no models at {stage} stage")
        
        return None, None


@task
def switch_model_of_production(MLFLOW_TRACKING_URI, X_test, y_test, model_name, current_date):
    # mlflow.set_tracking_uri("http://127.0.0.1:5000/")

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow_client = MlflowClient(MLFLOW_TRACKING_URI)

    staging_model, staging_version = load_model(MLFLOW_TRACKING_URI, model_name, stage = "Staging")
    if staging_model:
        staging_test_prediction = staging_model.predict(X_test)
        rmse_staging = mean_squared_error(staging_test_prediction, y_test, squared=False)
        print(staging_version, rmse_staging)
    else:
        rmse_staging = np.inf

    production_model, production_version = load_model(MLFLOW_TRACKING_URI, model_name, stage = "Production")
    if production_model:
        production_test_prediction = production_model.predict(X_test)
        rmse_production = mean_squared_error(production_test_prediction, y_test, squared=False)
        print(production_version, rmse_production)

    else:
        rmse_production = np.inf

    if rmse_staging <= rmse_production:
        print(f"$$$ Need to switch models. Version {staging_version} is better than {production_version} $$$")
        return staging_version
        
    else:
        print(f"$$$ No need to switch models. Version {production_version} is the best $$$")
        return None


@flow(task_runner = SequentialTaskRunner())
def main(current_date = "2015-6-21", periods = 5):
    
    MLFLOW_TRACKING_URI = 'sqlite:///../mlops-project.db'
    best_models_experiment = "Auction-car-prices-best-models"
    model_name = "Auction-car-prices-prediction"

    train_data = load_data(current_date = current_date, periods = periods)
    X, y = na_filter(train_data)

    test_data = load_data(current_date = current_date)
    X_test, y_test = na_filter(test_data)

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

    print("$$$ Training preprocessor... $$$")
    train, preprocessor = prepare_features(X_train, preprocessor = None )
    valid, _  = prepare_features(X_valid, preprocessor)

    print("$$$ Initializing parameters for baseline models... $$$")
    models = {
        LinearRegression: {
            "fit_intercept": hp.choice("fit_intercept", ('True', 'False'))
            },
        Ridge: {"alpha": hp.loguniform("alpha", -5, 5),
                "fit_intercept": hp.choice("fit_intercept", ('True', 'False'))
            },
        # RandomForestRegressor: {
        #         'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        #         'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        #         'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        #         'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        #         'random_state': 42
        #         },
        # XGBRegressor: {
        #         'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        #         'learning_rate': hp.loguniform('learning_rate', -3, 0),
        #         'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        #         'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        #         'max_child_weight': hp.loguniform('max_child_weight', -1, 3),
        #         'num_boost_rounds': 100,
        #         # 'early_stopping_rounds': 20,
        #         'objective': 'reg:squarederror',
        #         'seed': 42,
        #         }
        }

    best_models = params_search(
        MLFLOW_TRACKING_URI,
        train, valid,
        y_train, y_valid,
        current_date,
        models)

    # print("$$$ Best params $$$\n", best_models)

    best_pipelines = train_best_models(
        MLFLOW_TRACKING_URI,
        best_models_experiment,
        train, y_train,
        X_valid, y_valid,
        X_test, y_test,
        preprocessor,
        models = models
        )

    registered_model = model_to_registry(MLFLOW_TRACKING_URI, best_models_experiment, model_name)
    # print(registered_model.run_id, registered_model.version)

    model_promotion(
        MLFLOW_TRACKING_URI,
        current_date,
        model_name,
        registered_model_version=registered_model.version,
        to_stage = "Staging"
        )

    switch_to_version = switch_model_of_production(MLFLOW_TRACKING_URI, X_test, y_test, model_name, current_date)

    if switch_to_version:
        pass
        model_promotion(
            MLFLOW_TRACKING_URI,
            current_date = current_date,
            model_name = model_name,
            registered_model_version = switch_to_version,
            to_stage="Production"
            )
    else:
        print("$$$ Current is OK $$$")

main()
# Deployment(flow = main,
#                 name = 'MLOps-project-auction-car-prices',
#                 schedule = IntervalSchedule(interval = timedelta(minutes=5)), 
#                 flow_runner = SubprocessFlowRunner(),
#                 tags = ['ml-cpu'])

00:01:49.528 | INFO    | prefect.engine - Created flow run 'victorious-shrew' for flow 'main'
00:01:49.701 | INFO    | Flow run 'victorious-shrew' - Created task run 'load_data-2ff00c39-0' for task 'load_data'
00:01:49.702 | INFO    | Flow run 'victorious-shrew' - Executing 'load_data-2ff00c39-0' immediately...


Getting TRAIN data for 2014-12 period
Getting TRAIN data for 2015-1 period
Getting TRAIN data for 2015-2 period
Getting TRAIN data for 2015-3 period
Getting TRAIN data for 2015-4 period


00:01:52.823 | INFO    | Task run 'load_data-2ff00c39-0' - Finished in state Completed()
00:01:52.857 | INFO    | Flow run 'victorious-shrew' - Created task run 'na_filter-b6150c66-0' for task 'na_filter'
00:01:52.858 | INFO    | Flow run 'victorious-shrew' - Executing 'na_filter-b6150c66-0' immediately...
00:01:54.184 | INFO    | Task run 'na_filter-b6150c66-0' - Finished in state Completed()
00:01:54.212 | INFO    | Flow run 'victorious-shrew' - Created task run 'load_data-2ff00c39-1' for task 'load_data'
00:01:54.213 | INFO    | Flow run 'victorious-shrew' - Executing 'load_data-2ff00c39-1' immediately...


Getting TEST data for 2015-5 period


00:01:54.617 | INFO    | Task run 'load_data-2ff00c39-1' - Finished in state Completed()
00:01:54.645 | INFO    | Flow run 'victorious-shrew' - Created task run 'na_filter-b6150c66-1' for task 'na_filter'
00:01:54.646 | INFO    | Flow run 'victorious-shrew' - Executing 'na_filter-b6150c66-1' immediately...
00:01:54.823 | INFO    | Task run 'na_filter-b6150c66-1' - Finished in state Completed()
00:01:55.082 | INFO    | Flow run 'victorious-shrew' - Created task run 'prepare_features-fbf67cd7-0' for task 'prepare_features'
00:01:55.083 | INFO    | Flow run 'victorious-shrew' - Executing 'prepare_features-fbf67cd7-0' immediately...
2022/08/22 00:01:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '7dadf36f981f4cc5a8b4be4d04efdc41', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/08/22 00:01:55 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[(

...

2 1622.8923627321776
$$$ There are no models at Production stage
$$$ Need to switch models. Version 2 is better than None $$$


00:02:27.759 | INFO    | Task run 'model_promotion-877fbdb9-1' - Finished in state Completed()


$$$ Model Auction-car-prices-prediction version 2 was promoted to Production 2015-6-21 $$$


00:02:31.876 | INFO    | Flow run 'victorious-shrew' - Finished in state Completed('All states completed.')


[Completed(message=None, type=COMPLETED, result=      year    make                model   trim   body transmission  \
 0     2015     Kia              Sorento     LX    SUV    automatic   
 1     2015     Kia              Sorento     LX    SUV    automatic   
 2     2014     BMW  6 Series Gran Coupe   650i  Sedan    automatic   
 3     2015  Nissan               Altima  2.5 S  Sedan    automatic   
 4     2014     BMW                   M5   Base  Sedan    automatic   
 ...    ...     ...                  ...    ...    ...          ...   
 1445  2004    Jeep              Liberty  Sport    SUV          NaN   
 1446  2001  Nissan               Maxima     SE  Sedan    automatic   
 1447  1997   Lexus               ES 300   Base  Sedan    automatic   
 1448  2015    Ford               Escape     SE    SUV    automatic   
 1449  2014  Toyota              Corolla     LE  Sedan    automatic   
 
                     vin state  condition  odometer   color interior  \
 0     5xyktca69fg566472   

Getting TRAIN data for 2014-12 period
Getting TRAIN data for 2015-1 period
Getting TRAIN data for 2015-2 period
Getting TRAIN data for 2015-3 period
Getting TRAIN data for 2015-4 period
Getting TEST data for 2015-5 period


2022/08/21 23:56:49 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'aeb040de567e4758860b69276ff450fc', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/08/21 23:56:50 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('filler', ColumnTransformer(transformers=[('num_imputer', SimpleImputer(),
                                 ['condition', 'odometer', 'mmr']),
                                ('cat_imputer',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['body', 'transmission']),
                                ('cat_constant',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['color', 'int...`
2022/08/21 23:56:50 WARNING mlflow.utils: Truncated the value of the key `filler`. Truncated value: `ColumnTransformer(transformers=[('nu

$$$ Training preprocessor... $$$


2022/08/21 23:56:54 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2022/08/21 23:56:54 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not specify a `predict` function, which is required in order to infer the signature


$$$ Initializing parameters for baseline models... $$$
$$ Serching for the best parameters... $$          
$$ Predicting on the valid dataset... $$           
$$ RMSE on valid $$                                
1618.6086765949942                                   
$$ Serching for the best parameters... $$                                    
$$ Predicting on the valid dataset... $$                                     
$$ RMSE on valid $$                                                          
1618.6086765949942                                                             
100%|██████████| 2/2 [00:03<00:00,  1.78s/trial, best loss: 1618.6086765949942]
$$$ Best model $$$
 LinearRegression(fit_intercept='True')
$$ Serching for the best parameters... $$          
$$ Predicting on the valid dataset... $$           
$$ RMSE on valid $$                                
1765.2211391489516                                   
$$ Serching for the best parameters... $$                               

...


$$$ Best model $$$
 Ridge(alpha=3.4390426908768585, fit_intercept='False')
$$$ Training LinearRegression with best params $$$
$$$ LinearRegression MODEL was saved as a RUN of Auction-car-prices-best-models $$$
$$$ Training Ridge with best params $$$


Successfully registered model 'Auction-car-prices-prediction'.
2022/08/21 23:57:17 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Auction-car-prices-prediction, version 1


$$$ Ridge MODEL was saved as a RUN of Auction-car-prices-best-models $$$
$$$ Registering model Auction-car-prices-prediction $$$
$$$ Model RUN_ID 5b309866744c45e8978244b880c8c63c was registered as version 1 at None stage $$$
$$$ Model Auction-car-prices-prediction version 1 was promoted to Staging 2015-6-21 $$$


Created version '1' of model 'Auction-car-prices-prediction'.


1 1622.8923627321776
$$$ There are no models at Production stage
$$$ Need to switch models. Version 1 is better than None $$$
$$$ Model Auction-car-prices-prediction version 1 was promoted to Production 2015-6-21 $$$


In [8]:
# loaded_model = mlflow.pyfunc.load_model(model_uri=model_uri)
# print(loaded_model)
# loaded_model.predict(X_test)

model_name = "Auction-car-prices-prediction"
stage = 'Production'
run_id = "9c07c4e0556746a89adee76fe681290d"
production_model = mlflow.pyfunc.load_model(model_uri=f'models:/{model_name}/{stage}')
# production_model = mlflow.pyfunc.load_model(model_uri=f'runs:/{run_id}/model/')

MlflowException: The following failures occurred while downloading one or more artifacts from s3://kkr-mlops-zoomcamp/mlflow-artifacts/10/9725143bd7aa4114b7c1e21fe11e1d8d/artifacts/full-pipeline: {'': "ClientError('An error occurred (404) when calling the HeadObject operation: Not Found')"}